# Summarize Vulnerabilities

Import the Python dependencies needed for this analysis:

In [1]:
import json
import pathlib
import typing

from icecream import ic
from pydantic import BaseModel
import pandas as pd

Define a `pydantic` data class to represent the key, actionable information for each reported vulnerability:

In [2]:
class Vulnerabilty (BaseModel):
    id: str
    severity: str
    description: str
    url: str
    kind: str
    artifact: str
    version: str
    fix: str
    location: typing.List[str]

Load the JSON output from the `grype` vulnerability scan:

In [3]:
cve_path = pathlib.Path("cve.json")

with open(cve_path, "r") as fp:
    data = json.load(fp)

Group the vulnerabilities by their unique `id` values:

In [4]:
vuln_set: dict = {}

for item in data["matches"]:
    #ic(item)
    id = item["vulnerability"]["id"]
    
    if id not in vuln_set:
        vuln_set[id] = Vulnerabilty(
            id = item["vulnerability"]["id"],
            description = item["vulnerability"]["description"],
            severity = item["vulnerability"]["severity"],
            fix = item["vulnerability"]["fix"]["versions"][0],
            url = item["vulnerability"]["urls"][0],
            kind = item["artifact"]["type"],
            artifact = item["artifact"]["name"],
            version = item["artifact"]["version"],
            location = [ item["artifact"]["locations"][0]["path"] ],
        )
    else:
        vuln_set[id].location.append(item["artifact"]["locations"][0]["path"])

Construct a `pandas` dataframe to sort these by severity:

In [5]:
df = pd.DataFrame([
        item.__dict__
        for item in vuln_set.values()
    ])

columns = [ "severity", "kind", "artifact", "version", "fix", "description", "location", "id", "url" ]
df = df.reindex(columns=columns)
df = df.sort_values(by=[ "severity", "kind", "artifact", "version" ])

df_report = df.drop("url", axis=1)
df_report.head()

,severity,kind,artifact,version,fix,description,location,id


Report the actionable vulnerabilities and their suggested fixes:

In [6]:
df_report

,severity,kind,artifact,version,fix,description,location,id


In [7]:
with open(pathlib.Path("todo.json"), "w") as fp:
    data = json.loads(df.to_json(orient="records"))
    fp.write(json.dumps(data, indent=4, sort_keys=True))